# Momento Vector Index (MVI)

>[MVI](https://gomomento.com)：您数据最高效、最易用的无服务器向量索引。开始使用 MVI，只需注册一个账户即可。无需处理基础设施、管理服务器或担心扩展问题。MVI 是一项可自动扩展以满足您需求的のでサービス。

要注册并访问 MVI，请访问 [Momento Console](https://console.gomomento.com)。

# 设置

## 安装先决条件

您需要：
- [`momento`](https://pypi.org/project/momento/) 包，用于与 MVI 交互，以及
- `openai` 包，用于与 OpenAI API 交互。
- `tiktoken` 包，用于对文本进行分词。

In [ ]:
%pip install --upgrade --quiet  momento langchain-openai langchain-community tiktoken

## 输入 API 密钥

In [3]:
import getpass
import os

### Momento：用于索引数据

请访问 [Momento Console](https://console.gomomento.com) 获取您的 API 密钥。

In [ ]:
if "MOMENTO_API_KEY" not in os.environ:
    os.environ["MOMENTO_API_KEY"] = getpass.getpass("Momento API Key:")

### OpenAI：用于文本嵌入

In [18]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# 加载你的数据

这里我们使用 Langchain 的范例数据集，即“国情咨文”。

首先，我们加载相关的模块：

In [12]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import MomentoVectorIndex
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

然后我们加载数据：

In [24]:
loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
len(documents)

1

请注意，数据是一个大型文件，因此只有一个文档：

In [25]:
len(documents[0].page_content)

38539

因为这是一个大型文本文件，我们将其分割成块用于问答。这样，用户的问题将从最相关的块中得到解答。

In [26]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
len(docs)

42

# 索引你的数据

将数据建立索引就像实例化 `MomentoVectorIndex` 对象一样简单。在这里，我们使用 `from_documents` 助手来实例化和索引数据：

In [30]:
vector_db = MomentoVectorIndex.from_documents(
    docs, OpenAIEmbeddings(), index_name="sotu"
)

这通过您的 API 密钥连接到 Momento Vector Index 服务并索引数据。如果之前索引不存在，此过程会为您创建它。数据现在可供搜索。

# 查询你的数据

## 直接针对索引提问

查询数据的最直接方法是针对索引进行搜索。我们可以使用 `VectorStore` API 如下进行操作：

In [21]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_db.similarity_search(query)

In [22]:
docs[0].page_content

'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'

虽然这其中包含了关于 Ketanji Brown Jackson 的相关信息，但我们没有一个简洁、易于阅读的答案。我们将在下一节中解决这个问题。

## 使用 LLM 生成流利的答案

通过在 MVI 中索引数据，我们可以与任何利用向量相似性搜索的链进行集成。在此，我们使用 `RetrievalQA` 链来演示如何从索引数据中回答问题。

首先，我们加载相关模块：

In [27]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

然后我们实例化检索问答链：

In [31]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vector_db.as_retriever())

In [32]:
qa_chain({"query": "What did the president say about Ketanji Brown Jackson?"})

{'query': 'What did the president say about Ketanji Brown Jackson?',
 'result': "The President said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He described her as one of the nation's top legal minds and mentioned that she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans."}

# 下一步

好了！您现在已经索引了数据，并可以使用 Momento Vector Index 查询它。您可以使用相同的索引从任何支持向量相似性搜索的链中查询您的数据。

使用 Momento，您不仅可以索引向量数据，还可以缓存 API 调用并存储聊天消息历史记录。查看其他的 Momento langchain 集成以了解更多信息。

要了解有关 Momento Vector Index 的更多信息，请访问 [Momento 文档](https://docs.gomomento.com)。